In [28]:
import pandas as pd
import joblib

In [29]:
import os
os.listdir('../../../raw_data/indeed_proc/')

['ip_2021-03-03.joblib']

In [30]:
df = joblib.load('../../../raw_data/indeed_proc/ip_2021-03-03.joblib')
df = df[df.tag_language == 'en']
df.head()

,job_title,job_text,company,location,job_info,query_text,tag_language,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized
0,Data Scientist / Matching Engineer (m/w/d),You are responsible for improvement of Taledo’...,Taledo,Berlin,Data Scientist / Matching Engineer (m/w/d)\nTa...,data engineer,en,"[data, scientist, matching, engineer, mwd, tal...","[you, are, responsible, for, improvement, of, ...","[You, are, responsible, for, improvement, of, ...","[data, scientist, matching, engineer, mwd]"
3,Senior Software Engineer - Data Platform,We are looking for a Senior Software Engineer ...,Zalando SE,Berlin,Senior Software Engineer - Data Platform\nZala...,data engineer,en,"[senior, software, engineer, data, platform, z...","[we, are, looking, for, a, senior, software, e...","[We, are, looking, for, a, Senior, Software, E...","[senior, software, engineer, data, platform]"
5,Senior Software Engineer - Data Platform,We are looking for a Senior Software Engineer ...,Zalando,Berlin,Senior Software Engineer - Data Platform\nZala...,data engineer,en,"[senior, software, engineer, data, platform, z...","[we, are, looking, for, a, senior, software, e...","[We, are, looking, for, a, Senior, Software, E...","[senior, software, engineer, data, platform]"
7,Senior Data Engineer (m/w/t),"As a member of the Data Engineering Team, you ...",Quandoo GmbH,Berlin,Senior Data Engineer (m/w/t)\nQuandoo GmbH17 B...,data engineer,en,"[senior, data, engineer, mwt, quandoo, gmbh, b...","[as, a, member, of, the, data, engineering, te...","[As, a, member, of, the, Data, Engineering, Te...","[senior, data, engineer, mwt]"
8,Data Engineer (w/m/d),We are digitty.io – an international start-up ...,digitty.io,Berlin,Data Engineer (w/m/d)\ndigitty.io - Berlin,data engineer,en,"[data, engineer, wmd, digittyio, berlin]","[we, are, digittyio, an, international, startu...","[We, are, digittyio, an, international, startu...","[data, engineer, wmd]"


### Load all files and gather all skills. 

In [31]:
csvs = [file for file in os.listdir() if file.endswith('.csv') 
                                      and '_multi' not in file]
csvs

['soft_skills.csv',
 'bus_skills.csv',
 'tech_adjectives.csv',
 'adjectives.csv',
 'programming.csv',
 'methodology.csv']

In [32]:
all_skills = []

for file in csvs:
    all_skills += list(pd.read_csv(file, header=None).iloc[:, 0])

all_skills = set(all_skills)

### Get all tokens with ocurrences >= 30.

In [33]:
from collections import Counter
token_count = Counter([token 
                       for lst in df.job_text_tokenized
                       for token in lst]).most_common()

over_30 = [tup[0] for tup in token_count if tup[1] >= 30]


### Remove tokens which are skills or are English stopwords. 

In [34]:
from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english')) 
new_tokens = [token for token in over_30
              if token not in stop_words
             and token not in all_skills]

In [35]:
len(new_tokens)

1024

In [42]:
pd.Series(new_tokens).to_csv('tokens_above_30.csv', index=None)